In [ ]:
# Azure Databricks has already the SparkSession class built-in , therefore no need to import it
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DateType, DoubleType, BooleanType

In [ ]:
spark

In [ ]:
# "dbutils" filesystem commands set for the next cell.
# Like bash, what's inside this mounting point directory ("/mnt/tokyoolympic" ).
# To do this step I need to authorize permissions to the app through the IAM section

In [ ]:
%fs  
ls "/mnt/tokyoolympic" 

path,name,size,modificationTime
dbfs:/mnt/tokyoolympic/raw-data/,raw-data/,0,1695730843000
dbfs:/mnt/tokyoolympic/transformed-data/,transformed-data/,0,1695730888000


In [ ]:
athletes = spark.read.format("csv").load("/mnt/tokyoolympic/raw-data/athletes.csv") # /mnt/tokyoolympic/<name>/<name of csv file>.csv")

In [ ]:
athletes.show() # head()

In [ ]:
athletes.printSchema()

In [ ]:
# headers being regarded as observations, to fix this utilize the option() method with the argument "header" being "true"

In [ ]:
athletes = spark.read.format("csv").option("header",
                                           "true")\
                                   .load("/mnt/tokyoolympic/raw-data/athletes.csv")

In [ ]:
athletes.show()

In [ ]:
entriesgender = spark.read.format("csv").option("header",
                                                "true")\
                                        .load("/mnt/tokyoolympic/raw-data/entriesgender.csv")

In [ ]:
entriesgender.show()

In [ ]:
entriesgender.printSchema()

In [ ]:
# 1st way to tranform data, manually change every column, this is to change string to integers with cast() method

In [ ]:

entriesgender = entriesgender.withColumn("Female",
                                         col("Female").cast(IntegerType()))\
                             .withColumn("Male",
                                         col("Male").cast(IntegerType()))\
                             .withColumn("Total",
                                         col("Total").cast(IntegerType()))

In [ ]:
entriesgender.printSchema()

In [ ]:
# 2nd way,  inferschema method to autodetect the schema from the csv file

In [ ]:
entriesgender = spark.read.format("csv").option("header",
                                                "true")\
                                        .option("inferSchema",
                                                "true")\
                                        .load("/mnt/tokyoolympic/raw-data/entriesgender.csv")
                                           
                                


In [ ]:
entriesgender.printSchema() 

In [ ]:
athletes = spark.read.format("csv").option("header",
                                           "true")\
                                   .option("inferSchema",
                                           "true")\
                                   .load("/mnt/tokyoolympic/raw-data/athletes.csv")

coaches = spark.read.format("csv").option("header",
                                          "true")\
                                  .option("inferSchema",
                                          "true")\
                                  .load("/mnt/tokyoolympic/raw-data/coaches.csv")

entriesgender = spark.read.format("csv").option("header",
                                                "true")\
                                        .option("inferSchema",
                                                "true")\
                                        .load("/mnt/tokyoolympic/raw-data/entriesgender.csv")

medals = spark.read.format("csv").option("header",
                                         "true")\
                                 .option("inferSchema",
                                         "true")\
                                 .load("/mnt/tokyoolympic/raw-data/medals.csv")

teams = spark.read.format("csv").option("header",
                                        "true")\
                                .option("inferSchema",
                                        "true")\
                                .load("/mnt/tokyoolympic/raw-data/teams.csv")
                                                                                     

In [ ]:
# select top-ranked countries:
top_golds_countries = medals.orderBy("Rank by Total",
                                      ascending = True)\
                            .show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   3|               Japan|  27|    14|    17|   58|            5|
|   6|           Australia|  17|     7|    22|   46|            6|
|  10|               Italy|  10|    10|    20|   40|            7|
|   9|             Germany|  10|    11|    16|   37|            8|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
# select the team, # of golds and their rank pos.:
top_golds_countries = medals.orderBy("Gold",
                                      ascending = False)\
                            .select("Team_Country",
                                    "Gold",
                                    "Rank by Total")\
                            .show()


+--------------------+----+-------------+
|        Team_Country|Gold|Rank by Total|
+--------------------+----+-------------+
|United States of ...|  39|            1|
|People's Republic...|  38|            2|
|               Japan|  27|            5|
|       Great Britain|  22|            4|
|                 ROC|  20|            3|
|           Australia|  17|            6|
|         Netherlands|  10|            9|
|              France|  10|           10|
|             Germany|  10|            8|
|               Italy|  10|            7|
|                Cuba|   7|           18|
|         New Zealand|   7|           13|
|              Brazil|   7|           12|
|              Canada|   7|           11|
|             Hungary|   6|           13|
|   Republic of Korea|   6|           13|
|               Kenya|   4|           25|
|              Poland|   4|           19|
|      Czech Republic|   4|           23|
|              Norway|   4|           29|
+--------------------+----+-------

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn("Avg_Female",
                                                      entriesgender['Female'] / entriesgender['Total'])\
                                         .withColumn("Avg_Male",
                                                      entriesgender["Male"] / entriesgender["Total"])
average_entries_by_gender.show()
     

In [ ]:
athletes.write.option("header",
                      "true")\
              .csv("/mnt/tokyoolympic/transformed-data/athletes")

In [ ]:
# overwrite mode

In [ ]:
athletes.write.mode("overwrite")\
              .option("header",
                      "true")\
              .csv("/mnt/tokyoolympic/transformed-data/athletes")


In [ ]:
#repartition ( partittion data into differenr # of files)


In [ ]:
athletes.repartition(1).write.mode("overwrite")\
                             .option("header",
                                     "true")\
                             .csv("/mnt/tokyoolympic/transformed-data/athletes")


In [ ]:
coaches.repartition(1).write.mode("overwrite")\
                             .option("header",
                                     "true")\
                             .csv("/mnt/tokyoolympic/transformed-data/coaches")

entriesgender.repartition(1).write.mode("overwrite")\
                             .option("header",
                                     "true")\
                             .csv("/mnt/tokyoolympic/transformed-data/entriesgender")

medals.repartition(1).write.mode("overwrite")\
                             .option("header",
                                     "true")\
                             .csv("/mnt/tokyoolympic/transformed-data/medals")

teams.repartition(1).write.mode("overwrite")\
                             .option("header",
                                     "true")\
                             .csv("/mnt/tokyoolympic/transformed-data/teams")

     